In [17]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest


#####################################################
# Veri Seti Hikayesi
#####################################################

# Bir firmanın web site bilgilerini içeren bu veri setinde kullanıcıların gördükleri ve tıkladıkları
# reklam sayıları gibi bilgilerin yanı sıra buradan gelen kazanç bilgileri yer almaktadır.Kontrol ve Test
# grubu olmak üzere iki ayrı veri seti vardır. Bu veri setleriab_testing.xlsxexcel’ininayrı sayfalarında yer
# almaktadır. Kontrol grubuna Maximum Bidding, test grubuna AverageBiddinguygulanmıştır.

# impression: Reklam görüntüleme sayısı
# Click: Görüntülenen reklama tıklama sayısı
# Purchase: Tıklanan reklamlar sonrası satın alınan ürün sayısı
# Earning: Satın alınan ürünler sonrası elde edilen kazanç


In [18]:
# Adım 1:  ab_testing_data.xlsx adlı kontrol ve test grubu verilerinden oluşan veri setini okutunuz.
#Kontrol ve test grubu verilerini ayrı değişkenlere atayınız.

df1=pd.read_excel("/kaggle/input/ab-testing-data/ab_testing.xlsx",sheet_name="Control Group")
df2=pd.read_excel("/kaggle/input/ab-testing-data/ab_testing.xlsx",sheet_name="Test Group")

In [19]:
df1.head()

,Impression,Click,Purchase,Earning
0,82529.459271,6090.077317,665.211255,2311.277143
1,98050.451926,3382.861786,315.084895,1742.806855
2,82696.023549,4167.965750,458.083738,1797.827447
3,109914.400398,4910.882240,487.090773,1696.229178
4,108457.762630,5987.655811,441.034050,1543.720179


In [20]:
df1.isnull().sum()

Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64

In [21]:
df1.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.0,101711.449068,20302.157862,45475.942965,85726.690349,99790.701078,115212.816543,147539.336329
Click,40.0,5100.657373,1329.985498,2189.753157,4124.304129,5001.220602,5923.803596,7959.125069
Purchase,40.0,550.894059,134.108201,267.028943,470.095533,531.206307,637.957088,801.795020
Earning,40.0,1908.568300,302.917783,1253.989525,1685.847205,1975.160522,2119.802784,2497.295218


In [22]:
df2.head()

,Impression,Click,Purchase,Earning
0,120103.503796,3216.547958,702.160346,1939.611243
1,134775.943363,3635.082422,834.054286,2929.405820
2,107806.620788,3057.143560,422.934258,2526.244877
3,116445.275526,4650.473911,429.033535,2281.428574
4,145082.516838,5201.387724,749.860442,2781.697521


In [23]:
df2.isnull().sum()

Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64

In [24]:
df2.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.0,120512.411758,18807.448712,79033.834921,112691.970770,119291.300775,132050.578933,158605.920483
Click,40.0,3967.549761,923.095073,1836.629861,3376.819024,3931.359804,4660.497911,6019.695079
Purchase,40.0,582.106097,161.152513,311.629515,444.626828,551.355732,699.862360,889.910460
Earning,40.0,2514.890733,282.730852,1939.611243,2280.537426,2544.666107,2761.545405,3171.489708


#####################################################
# Görev 2:  A/B Testinin Hipotezinin Tanımlanması
#####################################################

# Adım 1: Hipotezi tanımlayınız.

#H0: M1 = M2 (İki grup arasında fark yoktur.)   p-value < 0.05'ten küçük ise H0 Red.

#H1: M1 != M2 (İki grup arasında fark vardır.)  p-value > 0.05'ten büyük ise H0 Reddedilemez.

# Adım 2: Varsayım Kontrolü 

#Normallik Varsayımı --> Shapiro Testi ile yapılır. --> Veri setinin normal dağılıma uygun olup olmadığını kontrol eder.
#Varyans Homojenliği --> Levene testi ile yapılır. --> İki grup arasında varyans eşitliğini kontrol eder.

# Adım 3: Hipotezlerin Kontrolü

#Varsayımlar sağlanıyorsa bağımsız iki örneklem t-testi(parametrik) ---> ttest-ind ile yapılır.
#Varsayımlar sağlanmıyorsa mannwitheyu testi ile yapılır(non-parametrik).
Buradan çıkan p-value değerine göre yorum yaparız.



In [25]:

#  Kontrol ve test grubu için purchase(kazanç) ortalamalarını analiz ediniz
print(df1["Purchase"].mean())
print(df2["Purchase"].mean())

550.8940587702316
582.1060966484677


#####################################################
# GÖREV 3: Hipotez Testinin Gerçekleştirilmesi
#####################################################

######################################################
# AB Testing (Bağımsız İki Örneklem T Testi)
######################################################


# Adım 1: Hipotez testi yapılmadan önce varsayım kontrollerini yapınız.Bunlar Normallik Varsayımı ve Varyans Homojenliğidir. 

# Kontrol ve test grubunun normallik varsayımına uyup uymadığını Purchase değişkeni üzerinden ayrı ayrı test ediniz. 


In [26]:

####----Normallik Varsayımı-Kontrol Grubu İçin----- ####
test_stat,pvalue=shapiro(df1["Purchase"])
print("Test Stat= %.4f,p-value=%.4f" % (test_stat,pvalue))
#p,0.05'ten büyük olduğu için H0:reddedilemez. Varsayımlar sağlanıyor.

Test Stat= 0.9773,p-value=0.5891


In [27]:
####----Normallik Varsayımı-Test Grubu İçin----- ####
test_stat,pvalue=shapiro(df2["Purchase"])
print("Test Stat= %.4f,p-value=%.4f" % (test_stat,pvalue))
#p,0.05'ten büyük olduğu için H0:reddedilemez. Varsayımlar sağlanıyor.

Test Stat= 0.9589,p-value=0.1541


In [28]:
#### ---Varyans Homojenliği --- ###
test_Stat,pvalue=levene(df2["Purchase"],df1["Purchase"])
print("Test Stat= %.4f,p-value=%.4f" % (test_stat,pvalue))

##0.05'ten büyük olduğu için H0 reddedilemez. Varyanslar homojendir.

Test Stat= 0.9589,p-value=0.1083


# Adım 2: Normallik Varsayımı ve Varyans Homojenliği sonuçlarına göre uygun testi seçiniz.

In [29]:
### Normallik varsayımını sağladığı için bağımsız iki örneklem t-testi(parametrik) uygulanmaktadır.
## Varyans homojenliği sağlandığı için equal_var değeri True olarak seçilir.

test_stat,pvalue=ttest_ind(df1["Purchase"],df2["Purchase"],
                        equal_var=True)
print("Test Stat= %.4f,p-value=%.4f" % (test_stat,pvalue))

## P , 0.05'ten büyük olduğu için H0:reddedilemez. Bu yüzden iki grup arasında fark yoktur.

Test Stat= -0.9416,p-value=0.3493
